In [1]:
"""
Description: Script to optimize Duca's currency mix.
Author: Jeroen van Dijk & Victor de Graaff
Date: 04-11-2020
Maintainer: Jeroen van Dijk & Victor de Graaff
Email: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl
Status: Dev
"""

"\nDescription: Script to optimize Duca's currency mix.\nAuthor: Jeroen van Dijk & Victor de Graaff\nDate: 04-11-2020\nMaintainer: Jeroen van Dijk & Victor de Graaff\nEmail: jeroen.vandijk@d-data.nl & victor.degraaff@d-data.nl\nStatus: Dev\n"

In [32]:
# Load common imports
%run ./CommonImports.ipynb

In [34]:
# Load common functions and currencies lists:
# - all_currencies
# - obsolete_currencies
# - p13_currencies
# - f_currencies
# - ff_currencies
# - currencies_per_continent
%run ./Utilities.ipynb

In [35]:
# Load loss functions
# - calculate_loss_function_around_one(weights)
# - calculate_loss_function_vs_t_minus_one(weights)
# - calculate_loss_function_vs_t_minus_one_with_balancing(weights)
# - calculate_loss_function_vs_t_minus_one_for_period(weights, max_date)
%run ./LossFunctions.ipynb

In [5]:
# Create dataframe with pre-defined base currency
df = create_original_df("EUR")

In [6]:
# Determine splits for dev- (train and test) and validation set
train_start = date(2005, 4, 1)
test_start = date(2012, 1, 1)
validation_start = date(2015, 1, 1)
validation_end = date(2020, 10, 1)

# Create dev (train and test) and validation set
train, test, dev, val = split_data(df, train_start, test_start, validation_start, validation_end)

In [7]:
# Create exchange table for each currency
exchange_table = dict()
for base_currency in all_currencies:
    exchange_table[base_currency] = 1/train[all_currencies].divide(train[base_currency], axis=0)

# Preview USD exchange table
exchange_table["USD"]

,TRY,NZD,SEK,HKD,NOK,CAD,HUF,USD,IDR,AUD,DKK,EUR,PHP,KRW,CNY,THB,RUB,ZAR,GBP,HRK,SGD,JPY,PLN,CHF,MYR,CZK,RON
date,,,,,,,,,,,,,,,,,,,,,,,,,,,
2005-04-01,0.73963,0.71051,0.14134,0.12823,0.15813,0.82353,0.00524,1.00000,0.00011,0.77123,0.17394,1.29590,0.01827,0.00099,0.12082,0.02554,0.03588,0.16175,1.88728,0.17460,0.60418,0.00932,0.31703,0.83461,0.26316,0.04315,0.35967
2005-04-02,0.73758,0.70945,0.14100,0.12822,0.15780,0.82270,0.00523,1.00000,0.00011,0.77012,0.17360,1.29337,0.01826,0.00099,0.12082,0.02546,0.03587,0.16153,1.88327,0.17424,0.60293,0.00930,0.31580,0.83284,0.26316,0.04307,0.35897
2005-04-03,0.73553,0.70838,0.14065,0.12822,0.15748,0.82187,0.00522,1.00000,0.00011,0.76901,0.17326,1.29083,0.01825,0.00099,0.12082,0.02537,0.03586,0.16132,1.87926,0.17387,0.60168,0.00928,0.31458,0.83106,0.26317,0.04299,0.35827
2005-04-04,0.73349,0.70731,0.14031,0.12821,0.15715,0.82104,0.00521,1.00000,0.00011,0.76790,0.17292,1.28830,0.01824,0.00099,0.12082,0.02529,0.03585,0.16110,1.87525,0.17351,0.60044,0.00926,0.31336,0.82929,0.26317,0.04291,0.35756
2005-04-05,0.73137,0.70431,0.13960,0.12821,0.15691,0.81681,0.00518,1.00000,0.00011,0.76441,0.17195,1.28100,0.01824,0.00098,0.12082,0.02522,0.03579,0.16014,1.87253,0.17266,0.60059,0.00921,0.31019,0.82427,0.26316,0.04271,0.35554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-27,0.52632,0.77395,0.14564,0.12859,0.16767,0.98101,0.00428,1.00000,0.00011,1.01609,0.17582,1.30690,0.02285,0.00087,0.15816,0.03181,0.03197,0.12258,1.56797,0.17385,0.77144,0.01284,0.29675,1.07035,0.31511,0.05068,0.30464
2011-12-28,0.52633,0.77766,0.14578,0.12858,0.16764,0.98530,0.00426,1.00000,0.00011,1.01989,0.17583,1.30740,0.02274,0.00087,0.15827,0.03171,0.03159,0.12334,1.56725,0.17367,0.77197,0.01288,0.29749,1.07199,0.31596,0.05064,0.30502
2011-12-29,0.52288,0.76652,0.14415,0.12866,0.16580,0.97725,0.00415,1.00000,0.00011,1.00703,0.17337,1.28890,0.02281,0.00087,0.15822,0.03147,0.03103,0.12265,1.54175,0.17100,0.76697,0.01286,0.29022,1.05769,0.31451,0.04975,0.29957


In [8]:
reference_date_exhange_rate = train[all_currencies].iloc[0]

In [26]:
def run_optimization(selected_currencies, 
                     max_iter=100, 
                     loss_function=calculate_loss_function_around_one):
    starting_point = np.ones(len(selected_currencies))/len(selected_currencies)
    bounds = [(0, 1) for _ in range(len(starting_point))]
    
    result = minimize(fun=loss_function, 
                      x0=starting_point, 
                      args=selected_currencies,
                      bounds=bounds,
                      options={"disp": True, 
                               "maxiter": max_iter})
    
    output = pd.Series(result.x/result.x.sum(), index=reference_date_exhange_rate.loc[selected_currencies].index)
    output = output.sort_values(ascending=False)
        
    return output

In [27]:
def try_currencies(currencies_to_try, currencies):
    for currency in currencies_to_try:
        if currency in currencies:
            return None, 1

    print(f"Evaluating {currencies} + {currencies_to_try}")
    new_mix = run_optimization(selected_currencies=currencies + currencies_to_try, 
                               max_iter=1, 
                               loss_function=calculate_loss_function_around_one)

    new_score = calculate_loss_function_around_one(new_mix, currencies + currencies_to_try)

    return new_mix, new_score

In [11]:
best_score = 1
last_score = 1
currencies = []

uncoupled_currencies = [[c] for c in all_currencies if c != "HKD"]
uncoupled_currencies.sort()

currency_pairs = [[c1[0], c2[0]] for c1 in uncoupled_currencies for c2 in uncoupled_currencies if c1 > c2]
currency_pairs.sort()

while best_score == 1 or currencies_to_add is not None:
    print(f"Attempting to improve set, starting from: {currencies}")
    currencies_to_add = None
    
    for currencies_to_try in uncoupled_currencies + currency_pairs:
        new_mix, new_score = try_currencies(currencies_to_try, currencies)

        if new_score < .9995 * last_score:
            if new_score < best_score:
                best_score = new_score
                currencies_to_add = currencies_to_try

                print(f"New best score found: {new_score}")
                print(new_mix)
            elif last_score < 1:
                print(f"Better than last score, but not better than best: {new_score}")
                print(new_mix)
    
    last_score = best_score
    
    if currencies_to_add is not None:
        currencies += currencies_to_add
        
print(f"Done. Best set: {currencies}")

Attempting to improve set, starting from: []
Evaluating [] + ['AUD']
New best score found: 0.02209449111512458
AUD   1.00000
dtype: float64
Evaluating [] + ['CAD']
New best score found: 0.01815195401505861
CAD   1.00000
dtype: float64
Evaluating [] + ['CHF']
Evaluating [] + ['CNY']
Evaluating [] + ['CZK']
Evaluating [] + ['DKK']
New best score found: 0.009534298316224626
DKK   1.00000
dtype: float64
Evaluating [] + ['EUR']
Evaluating [] + ['GBP']
Evaluating [] + ['HRK']
Evaluating [] + ['HUF']
Evaluating [] + ['IDR']
Evaluating [] + ['JPY']
Evaluating [] + ['KRW']
Evaluating [] + ['MYR']
Evaluating [] + ['NOK']
Evaluating [] + ['NZD']
Evaluating [] + ['PHP']
Evaluating [] + ['PLN']
Evaluating [] + ['RON']
Evaluating [] + ['RUB']
Evaluating [] + ['SEK']
Evaluating [] + ['SGD']
Evaluating [] + ['THB']
Evaluating [] + ['TRY']
Evaluating [] + ['USD']
New best score found: 0.008417071281971677
USD   1.00000
dtype: float64
Evaluating [] + ['ZAR']
Evaluating [] + ['CAD', 'AUD']
Evaluating [] 

In [14]:
mix6 = run_optimization(selected_currencies=['USD', 'DKK', 'KRW', 'JPY', 'GBP', 'CNY'], 
                                   max_iter=100, 
                                   loss_function=calculate_loss_function_around_one)

In [15]:
calculate_loss_function_around_one(mix6.values, mix6.index) / 0.007002770452103174

1.0019627412414622

In [16]:
# Create initial loss function values,
# to find find most/least stable currencies
results = pd.DataFrame()

# Calculate loss function score per currency
for base_currency in all_currencies:
    for currency in all_currencies:
        if currency == base_currency:
            continue
            
        rates = dev[base_currency] / dev[currency]
        normalized_rates = rates / rates.iloc[0]
        
        deviation = np.log(normalized_rates)
        loss_function_score = (deviation**2).mean()

        results = results.append({"currency": currency,
                                  "base_currency": base_currency,
                                  "loss_function_score": loss_function_score},
                                 ignore_index=True)
    
results = results.groupby("currency").median()
results = results.sort_values("loss_function_score", ascending=True)

,loss_function_score
currency,
CAD,0.01460
MYR,0.01597
NOK,0.01745
HRK,0.02022
SEK,0.02037
CZK,0.02069
EUR,0.02119
DKK,0.02121
USD,0.02143


In [54]:
# Select currencies where loss function should be calculated on:
# we exclude 2 least stable currencies
currencies_loss_function = list(results[:-2].index)

In [55]:
# Loss function is edited:
# not all currencies are used to calculate the loss function score

def calculate_loss_function_around_one(weights, selected_currencies):
    weights = weights / weights.sum()
    amounts = pd.Series(weights, index=selected_currencies) * reference_date_exhange_rate

    loss_function_score = 0

    for base_currency in currencies_loss_function:
        amounts_through_time = amounts * exchange_table[base_currency]
        normalized_amounts_through_time = amounts_through_time.sum(axis=1) / reference_date_exhange_rate[base_currency]
        
        deviation = np.log(normalized_amounts_through_time)
        loss_function_score += trading_volumes[base_currency] * (deviation**2).mean()
    
    return loss_function_score

In [ ]:
best_score = 1
last_score = 1
currencies = []

uncoupled_currencies = [[c] for c in all_currencies if c != "HKD"]
uncoupled_currencies.sort()

currency_pairs = [[c1[0], c2[0]] for c1 in uncoupled_currencies for c2 in uncoupled_currencies if c1 > c2]
currency_pairs.sort()

while best_score == 1 or currencies_to_add is not None:
    print(f"Attempting to improve set, starting from: {currencies}")
    currencies_to_add = None
    
    for currencies_to_try in uncoupled_currencies + currency_pairs:
        new_mix, new_score = try_currencies(currencies_to_try, currencies)

        if new_score < .9995 * last_score:
            if new_score < best_score:
                best_score = new_score
                currencies_to_add = currencies_to_try

                print(f"New best score found: {new_score}")
                print(new_mix)
            elif last_score < 1:
                print(f"Better than last score, but not better than best: {new_score}")
                print(new_mix)
    
    last_score = best_score
    
    if currencies_to_add is not None:
        currencies += currencies_to_add
        
print(f"Done. Best set: {currencies}")

Attempting to improve set, starting from: []
Evaluating [] + ['AUD']
New best score found: 0.021259912367901715
AUD   1.00000
dtype: float64
Evaluating [] + ['CAD']
New best score found: 0.017334239923574413
CAD   1.00000
dtype: float64
Evaluating [] + ['CHF']
Evaluating [] + ['CNY']
Evaluating [] + ['CZK']
Evaluating [] + ['DKK']
New best score found: 0.009018412226329936
DKK   1.00000
dtype: float64
Evaluating [] + ['EUR']
Evaluating [] + ['GBP']
Evaluating [] + ['HRK']
Evaluating [] + ['HUF']
Evaluating [] + ['IDR']
Evaluating [] + ['JPY']
Evaluating [] + ['KRW']
Evaluating [] + ['MYR']
Evaluating [] + ['NOK']
Evaluating [] + ['NZD']
Evaluating [] + ['PHP']
Evaluating [] + ['PLN']
Evaluating [] + ['RON']
Evaluating [] + ['RUB']
Evaluating [] + ['SEK']
Evaluating [] + ['SGD']
Evaluating [] + ['THB']
Evaluating [] + ['TRY']
Evaluating [] + ['USD']
New best score found: 0.008085068258062714
USD   1.00000
dtype: float64
Evaluating [] + ['ZAR']
Evaluating [] + ['CAD', 'AUD']
Evaluating [